In [1]:
import pandas as pd
import os 

In [31]:
from glob import glob

In [36]:
len(glob("../data/audiofiles/*.wav"))

431

In [18]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("../data/audiofiles/"):
    f.extend([f.replace(".wav", ".mp4") for f in filenames])
    #print(f.extend(filenames))
filenames =f

In [43]:
# Information about recordings with the recording ID and the original filenames
grabs=pd.read_csv("../data/grabs_v1.csv")
grabs2=pd.read_csv("../data/grabs_v2.csv")

In [44]:
def find_file(fil, grabs, grabs2):
    try:
        id_grab=int(fil.split("_")[0])
    except ValueError:
        return None
    filt=grabs[grabs.id_grabacion==id_grab]
    cand1="Not found"
    cand2="Not found"
    
    if len(filt)>0:
        cand1=".".join(filt.iloc[0]["nombre_original"].split(".")[:-1])
        prefijo1=filt.iloc[0]["prefijo"]
    filt=grabs2[grabs2.id_grabacion==id_grab]
    if len(filt)>0:
        cand2=".".join(filt.iloc[0]["nombre_original"].split(".")[:-1])
        prefijo2=filt.iloc[0]["prefijo"]
    if len(cand2)<2: cand2="Not found"
    if len(cand1)<2: cand1="Not found"
    filf=fil
    #print(filf)
    #print(cand1)
    #print(cand2)
    #print(filf.find(cand1))
    #print(filf.find(cand2))
    exito=0
    if filf.find(cand1)>=0 and filf.find(cand2)==-1:
        exito=1
        return prefijo1
    if filf.find(cand1)==-1 and filf.find(cand2)>=0:
        exito=1
        return prefijo2
        
    #print(exito)

In [23]:
dict_files={}
for f in filenames:
    dict_files[f]=find_file(f, grabs, grabs2)

In [45]:
len(dict_files)

435

In [49]:
len([f for f, k in dict_files.items() if k])

430

In [25]:
bt=pd.read_csv("../data/base_todos_2.csv")

/u/tsouth/python/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (132) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [59]:
", ".join(bt.columns)

'Unnamed: 0, Unnamed: 0.1, id_estudiante, tel_mas_confiable, id_tutor, prefijo, completado, completo_mate, completo_bienestar, completo_opinion, completo_español, completo_español_inicial, completo_mate_inicial, tratamiento, score_mat_inicial_x, score_mat_inicial_abiertas_x, score_mat_inicial_mult_x, grado_x, score_mat_inicial_y, score_mat_inicial_abiertas_y, score_mat_inicial_mult_y, grado_y, score_mat_inicial, score_mat_inicial_abiertas, score_mat_inicial_mult, grado, score_mat_final, score_mat_final_abiertas, score_cree, score_mat_final_mult, Unnamed: 0_x, score_esp_final, Por favor escribe un párrafo sobre quién es tu modelo a seguir y porque.\xa0, Por favor escribe un par sobre que quieres hacer cuando seas grande., Por favor escribe un par sobre que quieres hacer cuando seas grande.\nPor ejemplo\nCuando sea grande voy a tener un gato y un perro y voy a vivir en la ciudad de Tijuana cerca de mis papás y también voy a estudiar la carrera de Computación porque quiero ayudar a mi pap

In [ ]:
[~pd.isna(bt.score_mat_final)]

In [140]:
#base you can use
#prefijo: group kid was assigned to, matches with values on dict_files
#ChangeMathScore: change in math score
#score_mat_inicial: initial math score (raw)
#score_mat_final: final math score (raw)
#tel_mas_confiable: if >0, it a class identifier
#MathScore: Final math score (standardized at grade level)
#std_mat_inicial: Initial math score (standardized at grade level)
#tratamiento: 1: control, 2: math, 3: math + emotional learning, 4: math

some_vars=bt[["ChangeMathScore", "score_mat_inicial", "score_mat_final", "tel_mas_confiable", "MathScore", "std_mat_inicial", "tratamiento", "prefijo"]]
some_vars = some_vars[~some_vars['prefijo'].isna()]

In [141]:
filenames_df = pd.DataFrame(filenames, columns=['file'])
filenames_df['prefijo'] = filenames_df['file'].map(dict_files)
filenames_df = filenames_df.dropna()
some_vars = some_vars[~some_vars['prefijo'].isna()]
# some_vars.set_index('prefijo', inplace=True)


In [142]:
rows= []
for prefijo in filenames_df['prefijo']:
    if prefijo in some_vars.index:
        rows.append(some_vars.loc[prefijo].iloc[0])
    else:
        rows.append(pd.Series([None]*len(some_vars.columns), index=some_vars.columns))

In [169]:
varrow+filerow

(608,
 nan,
 nan,
 nan,
 6461171260,
 nan,
 nan,
 2.0,
 'JN137',
 1,
 '9885_5990_2021-05-11_tutoria9.mp4',
 'JN137')

In [168]:
len(list(some_vars.columns)+list(filenames_df.columns))

10

In [170]:
connected_rows = []
for varrow in some_vars.itertuples():
    for filerow in filenames_df.itertuples():
        if filerow.prefijo==varrow.prefijo:
            connected_rows.append(pd.Series(varrow+filerow, index = ['index1'] + list(some_vars.columns)+['index2']+list(filenames_df.columns)))

In [176]:
results_with_file = pd.concat(connected_rows, axis = 1).T

In [182]:
results_with_file = results_with_file.drop(['index1','index2'], axis=1)

In [183]:
results_with_file.to_csv('../data/reponse_var.csv', index=False)